In [47]:
import os
import shutil
import subprocess
import zipfile

from icecream import ic

from collage import make_diff_image

In [42]:
def extract(path_to_zip, target_dir):
    with zipfile.ZipFile(path_to_zip, 'r') as target_zip:
        target_zip.extractall(target_dir)
        
        
def find_main_tex(path):
    directory = os.path.join(os.getcwd(), path)
    tex_file = None
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tex'):
                tex_file = os.path.join(root, file)
                break
        if tex_file:
            break
    return tex_file


def get_image_paths(target_tex_file):
    image_paths = []
    with open(target_tex_file, 'r') as tex_file:
        for line in tex_file:
            if not line.strip().startswith('%') and 'includegraphics' in line:
                start_index = line.find('{') + 1
                end_index = line.find('}')
                image_path = line[start_index:end_index]
                image_paths.append(image_path)
    return image_paths

In [41]:
new_zip = 'new.zip'
old_zip = 'old.zip'

dir_new = 'new'
dir_old = 'old'

dir_new_full = os.path.join(os.getcwd(), dir_new)
dir_old_full = os.path.join(os.getcwd(), dir_old)

extract(new_zip, dir_new)
extract(old_zip, dir_old)

print(dir_new_full, '\n', dir_old_full)

/Users/ivan/Documents/GitHub/latexdiff-bot/test/new 
 /Users/ivan/Documents/GitHub/latexdiff-bot/test/old


In [43]:
old_tex_file = find_main_tex(folder_old)
new_tex_file = find_main_tex(folder_new)

print(old_tex_file, '\n', new_tex_file)

/Users/ivan/Documents/GitHub/latexdiff-bot/test/old/main.tex 
 /Users/ivan/Documents/GitHub/latexdiff-bot/test/new/main.tex


In [46]:
new_image_paths = get_image_paths(new_tex_file)
old_image_paths = get_image_paths(old_tex_file)

print(old_image_paths, '\n', new_image_paths)

['fig/figure1.png'] 
 ['fig/image.png', 'fig/E_2_2.png']


In [50]:
old_img_number = len(old_image_paths)
new_img_number = len(new_image_paths)

ic(
    old_img_number, new_img_number
); 

ic| old_img_number: 1, new_img_number: 2


In [ ]:


# Create the diff.tex file using latexdiff
diff_file = 'diff.tex'
latexdiff_command = f'latexdiff {old_tex_file} {new_tex_file} > new/{diff_file}'
subprocess.run(latexdiff_command, shell=True, check=True)

# Get the list of images used in the main LaTeX file
image_paths = []
with open(new_tex_file, 'r') as tex_file:
    for line in tex_file:
        if 'includegraphics' in line:
            start_index = line.find('{') + 1
            end_index = line.find('}')
            image_path = line[start_index:end_index]
            image_paths.append(image_path)

# Apply make_diff_image command to changed images
for image_path in image_paths:
    path_img_old = os.path.join(old_directory, image_path)
    path_img_new = os.path.join(new_directory, image_path)
    path_img_diff = os.path.join(new_directory, 'diff', image_path)
    make_diff_image(path_img_old, path_img_new, path_img_diff)

os.chdir('new')

# Compile the diff.tex file
pdflatex_command = f'latexmk -f -pdf {diff_file}'
subprocess.run(pdflatex_command, shell=True, check=True)

os.chdir('..')

shutil.move('new/diff.pdf', 'diff.pdf')

# Clean up temporary directories and files
shutil.rmtree('new')
shutil.rmtree('old')